In [51]:
from datasets import load_dataset, Value, Sequence, Features, Image

In [52]:
dataset = load_dataset('json', data_files={'train': 'train.json',
                                                'val': 'val.json'})

Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 327.94it/s]


Dataset json downloaded and prepared to /Users/jaydentran1909/.cache/huggingface/datasets/json/default-6e0a7ad532503484/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 315.40it/s]


In [88]:
ner_tags_list = ['ADDR', 'NUMBER_PREFIX', 'TITLE', 'PRODUCT_NAME_PREFIX',
       'PRODUCT_NAME', 'AMOUNT_PREFIX', 'AMOUNT', 'UNIT_PREFIX', 'UNIT',
       'UPRICE_PREFIX', 'UPRICE', 'SUB_TPRICE_PREFIX', 'SUB_TPRICE',
       'TAMOUNT_PREFIX', 'TAMOUNT', 'TPRICE_PREFIX', 'TPRICE',
       'RECEMONEY_PREFIX', 'RECEMONEY', 'OTHER', 'BILLID_PREFIX',
       'BILLID', 'DATETIME_PREFIX', 'DATETIME', 'CASHIER_PREFIX',
       'CASHIER', 'SHOP_NAME', 'PHONE_PREFIX', 'FPRICE_PREFIX', 'FPRICE',
       'REMAMONEY_PREFIX', 'REMAMONEY', 'PHONE', 'TDISCOUNT_PREFIX',
       'TDISCOUNT', 'ADDR_PREFIX', 'NUMBER', 'FAX_PREFIX', 'FAX',
       'UDISCOUNT_PREFIX', 'UDISCOUNT']
class_label = ClassLabel(names=ner_tags_list)

In [89]:
from datasets import Image
from PIL import Image as PImage
from functools import partial
import os
import numpy as np
# "image"
# "tokens"
# "bboxes"
# "ner_tags"
def mapping(examples, root_folder='images'):
    new_bboxes = []
    # map bboxes
    for size, bboxes in zip(examples['size'], examples['bboxes']):
        W, H = size
        tlx, tly, brx, bry = bboxes
        new_bboxes.append([int(tlx / W * 1000), int(tly / H * 1000), int(brx / W * 1000), int(bry / H * 1000)])
    examples['bboxes'] = new_bboxes
    
    # map image
    examples['image'] = PImage.open(
        os.path.join(root_folder, examples['image'])
    ).convert('RGB')
    
    # ner tags convert
    examples['ner_tags'] = [class_label.str2int(tag) for tag in examples['ner_tags']]
    
    return examples


new_dataset = dataset.map(mapping, remove_columns=['size'])

In [92]:
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

In [133]:
from datasets.features import ClassLabel

features = new_dataset["train"].features
column_names = new_dataset["train"].column_names
image_column_name = "image"
text_column_name = "tokens"
boxes_column_name = "bboxes"
label_column_name = "ner_tags"

image_feat_decoder = Image().decode_example
def prepare_examples(examples):
  images = image_feat_decoder(examples[image_column_name])
  words = examples[text_column_name]
  boxes = examples[boxes_column_name]
  word_labels = examples[label_column_name]
  encoding = processor(images, words, boxes=boxes, word_labels=word_labels,
                      truncation=True, padding="max_length")
  
  encoding['pixel_values'] = encoding['pixel_values'][0]
  return encoding

In [134]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

train_dataset = new_dataset["train"].map(
    prepare_examples,
    remove_columns=column_names
)

eval_dataset = new_dataset["val"].map(
    prepare_examples,
    remove_columns=column_names
)

In [135]:
train_dataset.set_format("torch")

In [136]:
import torch

example = train_dataset[0]
for k,v in example.items():
    print(k,v.shape)

input_ids torch.Size([512])
attention_mask torch.Size([512])
bbox torch.Size([512, 4])
labels torch.Size([512])
pixel_values torch.Size([3, 224, 224])


In [137]:
processor.tokenizer.decode(eval_dataset[0]["input_ids"])

'<s> PHÓ MÔ Tổ 7 Khu Tân lập 4 - P.Cẩm Thủy - Tp.Cẩm Phả - Quảng Ninh ĐT  0858.931.931 HÓA ĐƠN BÁN HÀNG Số HD130820-0006 Ban MANG VỀ [A] 13/08/2020 - 18:44 Khách hàng  SĐT Địa chỉ:  Đơn giá Trà chanh Up Sz  Trà quất Up Sz  Nước ép dưa hấu 20,000 20 000 20,000 SL 1 1 1 Thành tiền 20.000 20000 20.000 Cộng tiền hàng 60,000 Chiết khấu: o Tổng cộng 60,000 Tiền khách đưa  60.000 Tiên thừa  0 Sáu mươi nghìn đồng chẵn  Cám ơn và hẹn gặp lại!  Powered by POS365.VN</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [138]:
from datasets import load_metric

metric = load_metric("seqeval")

In [139]:
import numpy as np

return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [ner_tags_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [ner_tags_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [140]:
id2label = {k: v for k, v in enumerate(ner_tags_list)}
label2id = {v: k for k, v in enumerate(ner_tags_list)}

In [141]:
from transformers import LayoutLMv3ForTokenClassification

model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base",
                                                         id2label=id2label,
                                                         label2id=label2id)

loading configuration file config.json from cache at /Users/jaydentran1909/.cache/huggingface/hub/models--microsoft--layoutlmv3-base/snapshots/07c9b0838ccc7b49f4c284ccc96113d1dc527ff4/config.json
Model config LayoutLMv3Config {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "coordinate_size": 128,
  "eos_token_id": 2,
  "has_relative_attention_bias": true,
  "has_spatial_attention_bias": true,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "ADDR",
    "1": "NUMBER_PREFIX",
    "2": "TITLE",
    "3": "PRODUCT_NAME_PREFIX",
    "4": "PRODUCT_NAME",
    "5": "AMOUNT_PREFIX",
    "6": "AMOUNT",
    "7": "UNIT_PREFIX",
    "8": "UNIT",
    "9": "UPRICE_PREFIX",
    "10": "UPRICE",
    "11": "SUB_TPRICE_PREFIX",
    "12": "SUB_TPRICE",
    "13": "TAMOUNT_PREFIX",
    "14": "TAMOUNT",
    "15": "TPRICE_PREFIX",
    "16": "TPRICE",
    "17": "RECEMONEY_PREFIX",
    "18": "RECEMONEY",
    "19": "O

In [142]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test",
                                  max_steps=1000,
                                  per_device_train_batch_size=2,
                                  per_device_eval_batch_size=2,
                                  learning_rate=1e-5,
                                  evaluation_strategy="steps",
                                  eval_steps=100,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [143]:
train_dataset['pixel_values'][0].shape

torch.Size([3, 224, 224])

In [144]:
from transformers.data.data_collator import default_data_collator

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

max_steps is given, it will override any value given in num_train_epochs


In [145]:
trainer.train()

/Users/jaydentran1909/miniconda3/envs/bill_info_extract/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 372
  Num Epochs = 6
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 1000
  Number of trainable parameters = 125949097
  0%|          | 0/1000 [00:00<?, ?it/s]/Users/jaydentran1909/miniconda3/envs/bill_info_extract/lib/python3.10/site-packages/transformers/modeling_utils.py:763: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
  0%|          | 1/1000 [00:25<6:57:03, 25.05s/it]

KeyboardInterrupt: 